<a href="https://colab.research.google.com/github/Tiabet/Boat_Competition/blob/main/Kboat_kollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install peft transformers datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(i

In [2]:
from datasets import load_dataset

# Load JSONL dataset
dataset = load_dataset("json", data_files={"train": "/content/drive/MyDrive/data/train_double_smaller_v2.jsonl",
                                           "val": "/content/drive/MyDrive/data/val_double_smaller_v2.jsonl",
                                           "test":"/content/drive/MyDrive/data/test_double_smaller_v2.jsonl"})


Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(dataset["val"][0])  # Print the first training example

{'messages': [{'role': 'system', 'content': '당신은 유능한 승부예측 전문가입니다. 선수와 모터, 보트의 정보를 보고 우승여부를 예측해보세요.'}, {'role': 'user', 'content': '선수와 모터, 보트의 정보가 주어집니다. 아래 정보를 보고, 이 선수가 우승에 성공했을지 실패했을지 예측해보세요. 선수: 등급: A1, 성별: 0, 나이: 30-40, 최근6회차_평균착순점: 2.88, 최근6회차_평균득점: 2.86, 최근6회차_승률: 50.0, 최근6회차_연대율2: 63.6, 최근6회차_연대율3: 68.2, 최근6회차_평균ST: 0.2, 연간성적_평균착순점: 3.7800000000000002, 연간성적_연대율: 58.7, 평균사고점: 0.07, 전일성적: 6.0, 출주횟수: 43, 모터_평균착순점: 6.56, 모터_연대율2: 22.6, 모터_연대율3: 34.6, 보트_평균착순점: 5.29, 보트_연대율: 33.8, 중량부과여부: 0, 부과된중량: 0.0, PCA_코스성적_1: 65.37, PCA_코스성적_2: -1.71, 코스_성적: 81.36, 최근3경기_평균: 1.67'}, {'role': 'assistant', 'content': '우승'}]}


In [3]:
import torch
import torch.nn as nn

class CausalLMWithClassificationHead(nn.Module):
    def __init__(self, base_model, hidden_size):
        super(CausalLMWithClassificationHead, self).__init__()
        self.base_model = base_model
        self.classification_head = nn.Linear(hidden_size, 1)  # Binary classification head
        self.loss_fct = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Get the outputs from the base model and set output_hidden_states=True to get hidden states
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,  # Ensure hidden states are returned
            return_dict=True
        )

        # Extract the hidden states from the final layer
        hidden_states = outputs.hidden_states[-1]  # Shape: (batch_size, sequence_length, hidden_size)

        # Use the hidden state of the last token in the sequence for classification
        cls_token_state = hidden_states[:, -1, :]  # Shape: (batch_size, hidden_size)

        # Pass through the classification head to produce logits for binary classification
        logits = self.classification_head(cls_token_state).squeeze(-1)  # Shape: (batch_size,)

        # Compute the loss if labels are provided
        if labels is not None:
            labels = labels.float()  # Convert labels to float for BCEWithLogitsLoss
            labels = labels.view(-1)  # Ensure labels have the shape (batch_size,)
            loss = self.loss_fct(logits, labels)
            return loss, logits

        return logits


In [4]:
from peft import LoraConfig, get_peft_model

# Ensure the tokenizer uses EOS as PAD token
tokenizer.pad_token = tokenizer.eos_token

# Configure LoRA for efficient training
lora_config = LoraConfig(
    r=4,                           # Low-rank dimension
    lora_alpha=32,                   # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to specific layers
    lora_dropout=0.05,               # Dropout rate for LoRA
    bias="none",
    task_type="CAUSAL_LM"            # Task type (causal language model)
)

# Wrap the base model with LoRA
model_with_lora = get_peft_model(model, lora_config)

# Initialize the final model with the classification head
final_model = CausalLMWithClassificationHead(model_with_lora, hidden_size=model.config.hidden_size)


In [5]:
import torch

# Ensure the tokenizer uses EOS as PAD token
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Combine 'system' and 'user' message content into inputs
    inputs = [example[0]['content'] + " " + example[1]['content'] for example in examples['messages']]

    # Convert the assistant response ("우승" = 1, "실패" = 0) to labels
    labels = [1 if example[2]['content'] == "우승" else 0 for example in examples['messages']]

    # Tokenize the inputs, with padding and truncation
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=1024)

    # Add the labels to the tokenized inputs
    model_inputs["labels"] = torch.tensor(labels, dtype=torch.long)  # Ensure labels are long tensor for classification tasks

    return model_inputs

# Tokenize and preprocess the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/13950 [00:00<?, ? examples/s]

Map:   0%|          | 0/2952 [00:00<?, ? examples/s]

Map:   0%|          | 0/6102 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader

training_args = TrainingArguments(
    output_dir="./llama-lora-classification",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    save_steps=1000,
    logging_steps=100,
    fp16=True,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=final_model,                # Pass the final model with the classification head
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"].select(range(100)),
    tokenizer=tokenizer,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
100,2.918600,1.788650
200,2.073200,1.537684
300,1.773800,2.081429
400,2.018000,2.405378
500,1.811100,2.077349
600,2.046100,2.094198
700,1.741100,2.071954
800,1.788000,1.510736
900,1.851800,2.034291
1000,1.367700,2.206992


In [13]:
tokenized_datasets["val"].select(range(100))

Dataset({
    features: ['messages', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})